In [220]:
import torch
import os
import json
import glob
from collections import defaultdict
from os import path
from transformers import BertTokenizer

In [221]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

I0207 14:01:39.537658 140184527419200 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/shtoshni/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


### HTML Conversion

In [222]:
HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head><body>'

cluster_start_tag = '<div style="border:2px; display : inline; border-style:solid; padding: {}px; padding-right: 3px; padding-left: 3px">'
singleton_start_tag = '<div style="border:2px; display : inline; border-style:dotted; padding:{}px; padding-right: 3px; padding-left: 3px">'
end_tag = '</div>'

largest_padding = 13
padding_reduction = 3



In [223]:
def return_html(json_file):
    story_name = path.basename(story_file)
    print(story_name)
    
    with open(json_file) as f:
        doc = json.load(f)
    
    html_string = HTML_START + '<div style="line-height: 3">'
    
    all_subtokens = []
    for sentence in doc["tokenized_doc"]["sentences"]:
        all_subtokens.extend(sentence)
    
    all_tokens = []
    cur_subtoken_list = []
    cur_idx = 0
    for idx, subtoken_idx in enumerate(doc["tokenized_doc"]["subtoken_map"]):
        if subtoken_idx == cur_idx:
            if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                cur_subtoken_list.append(all_subtokens[idx])
        else:
#             print(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
            all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
            if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                cur_subtoken_list = [all_subtokens[idx]]
            else:
                cur_subtoken_list = []
            cur_idx = subtoken_idx
            
    
#     print(all_tokens[:30])
    if cur_subtoken_list != []:
        all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
    
#     print(len(doc["tokenized_doc"]["subtoken_map"]))
    print(len(all_tokens), doc["tokenized_doc"]["subtoken_map"][-1] + 1)
#     print(len(all_subtokens))
    
    ment_start_dict = defaultdict(list)
    ment_end_dict = defaultdict(list)
    cluster_idx_to_len = defaultdict(int)
    
    clusters = sorted(doc["clusters"], key=lambda x: x[0][0][0])
    
    for cluster_idx, ment_list in enumerate(clusters):
        for (ment_start, ment_end), string in ment_list:
#             try:
#                 assert (ment_end > ment_start)
#             except AssertionError:
#                 print((ment_start, ment_end), string)
            ment_start_dict[ment_start].append((ment_end, cluster_idx))
            ment_end_dict[ment_end].append((ment_start, cluster_idx))
            cluster_idx_to_len[cluster_idx] += 1
                        
    # Sort mentions with same mention start by later mention ends i.e. start with spans which are longer
    for ment_start in ment_start_dict:
        ment_start_dict[ment_start] = sorted(ment_start_dict[ment_start], key=lambda x: x[0], reverse=True)
        
    # Sort mentions with same mention end by later mention starts i.e. start with spans which are shorter
    for ment_end in ment_end_dict:
        ment_end_dict[ment_end] = sorted(ment_end_dict[ment_end], key=lambda x: x[0], reverse=True)
        
    active_clusters = 0
    for token_idx, token in enumerate(all_tokens):
        token_added = False
        if token == "\n":
            html_string += "<br/>\n" 
            continue
        if token_idx in ment_start_dict:
            for (_, cluster_idx) in ment_start_dict[token_idx]:
                prefix = cluster_start_tag
                if cluster_idx_to_len[cluster_idx] == 1:
                    prefix = singleton_start_tag
                html_string += prefix.format(largest_padding - active_clusters * padding_reduction)
                active_clusters += 1
            
            html_string += token + " "
            token_added = True
        
        if not token_added:
            html_string += token + " "

        if token_idx in ment_end_dict:
            for (_, cluster_idx) in ment_end_dict[token_idx]:
                html_string += "<sub>" + str(cluster_idx) + "</sub>" + end_tag + " "
                active_clusters -= 1
                assert (active_clusters >= 0)
    
    html_string += "</div></body></html>"
    return html_string

### Process all files to get HTML version of the data

In [224]:
story_file = "/home/shtoshni/Research/long-doc-coref/notebooks/ccarol_output.json"
book_html = return_html(story_file)

ccarol_output.json
41698 41698


In [225]:
html_file = "/home/shtoshni/Research/long-doc-coref/notebooks/ccarol_output.html"
with open(html_file, "w") as f:
    f.write(book_html)